In [ ]:
import pickle

# Embedding model trainings

### Embed company descriptions

#### Import Data

In [ ]:
with open("pipes/pipe4", "rb") as fp:   # Unpickling
    pipe4 = pickle.load(fp)

In [ ]:
pipe4[0]

[['le',
  'fourgon',
  'delivers',
  'stored',
  'drinks',
  'home',
  'order',
  'placed',
  'lefourgon.com',
  'beers',
  'juices',
  'sodas',
  'water',
  'milk',
  'wines',
  'soups',
  'spirits',
  'co.',
  'deliver',
  'home',
  'free',
  'charge',
  'chosen',
  'niche',
  'next',
  'visit',
  'collect',
  'empty',
  'bottles',
  'return',
  'washed',
  'producer',
  'reuse',
  'zerodechet']]

In [ ]:
from gensim.models import FastText, Word2Vec

#### Word2Vec

#### Fasttext

#### GloVe